# Let's make a log of the LUH16AB Binary components

| BTJD | Binary Component | Nod position| TESS Flux|
|--|--|--| -- |
|x | A | A | -- |
|y| B | A| -- |

The header info in the IGRINS Slit Viewing Camera was malformed.  Let's attempt to fix it so we can preserve our nice figures.

In [ ]:
import lightkurve as lk

In [ ]:
lk.__version__

In [ ]:
# %load /Users/obsidian/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
import os

In [ ]:
import aplpy

In [ ]:
aplpy.__version__

In [ ]:
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import (PercentileInterval, SqrtStretch,
                                   ImageNormalize)
from astropy.io import fits
from astropy.wcs import WCS
import montage_wrapper as montage

In [ ]:
from reproject import reproject_interp, reproject_exact

In [ ]:
pos = SkyCoord(ra='10 49 11.95', dec='-53 19 01.8', unit=(u.hourangle, u.deg), frame='icrs')

### What imaging do we want?

In [ ]:
rad_value = 60
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
def make_postage_stamp(im_in, output, hdu=None, vmin=None, vmax=None, radius=None):
    '''Make a postage stamp image'''
    fig = plt.figure(figsize=(5,5))
    if radius is None:
        radius = 60 * u.arcsecond
    if hdu is None:
        mosaic = aplpy.FITSFigure(im_in, north=True, figure = fig)
    else:
        mosaic = aplpy.FITSFigure(im_in,hdu=hdu, north=True, figure = fig)
    mosaic.recenter(pos.ra,pos.dec, radius=radius.to(u.degree).value)  # degrees
    if (vmin is not None) & (vmax is not None):
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='log',
                              vmin=vmin, vmax=vmax)
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='log',
                              vmin=vmin, vmax=vmax)        
    else:
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='log')
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='log')
    mosaic.tick_labels.set_font(size='small')
    #mosaic.tick_labels.set_xformat("hh:mm:ss")
    mosaic.set_theme('paper')

    aplpy.AxisLabels(mosaic).hide()
    aplpy.TickLabels(mosaic).hide()

    mosaic.add_scalebar(10.0 * u.arcsecond)
    #fig.scalebar.show(1.0/3600.0*4.0)  # length in degrees
    #mosaic.scalebar.set_length(4.0 * u.arcsecond)
    mosaic.scalebar.set_label("$10^{\prime\prime}$")
    #mosaic.scalebar.set_corner('bottom right')
    mosaic.scalebar.set_color('red')
    mosaic.scalebar.set_linestyle('solid')
    mosaic.scalebar.set_linewidth(5)  # points
    mosaic.scalebar.set_font(size=12, weight='medium', \
                          stretch='normal', family='sans-serif', \
                          style='normal', variant='normal')

    mosaic.save(output, dpi=300)

### IGRINS Slit Viewing Camera

In [ ]:
rad_value = 21
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
import glob

In [ ]:
fns = sorted(glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/**/SDCS*.fits'))

In [ ]:
fn = fns[0]
hdu = fits.open(fn)

In [ ]:
im_in = hdu
output = 'Luhman16_{}x{}arcsec_IGRINS_20210311.png'.format(diam, diam)

In [ ]:
hdu[1].header['CROTA1'] += 90
hdu[1].header['CROTA2'] += 90

In [ ]:
%%time
make_postage_stamp(im_in, output, hdu=1, vmin=150, vmax=6000, radius=rad)

In [ ]:
keep_keywords = ['MJD-OBS', 'TELRA', 'TELDEC', 'AMSTART']

In [ ]:
def get_header_info(fn, ext=1):
    df_out = pd.DataFrame()
    hdu = fits.open(fn)
    hdr = hdu[ext].header
    return {key:hdr[key] for key in keep_keywords}

In [ ]:
df_log = pd.DataFrame()
for i, fn in enumerate(fns):
    df = pd.DataFrame(get_header_info(fn), index=[i])
    basename = os.path.basename(fn)
    df['fn'] = basename
    df_log = df_log.append(df)

In [ ]:
redo = False

In [ ]:
if redo:
    df_log.to_excel('../../data/IGRINS/SVC_log_original.xlsx') # We changed the name of the labelled one.

Now we have to hand-label the components and nods with human visual inspection.

## Okay, now read in that labelled sheet

In [ ]:
df_labelled = pd.read_csv('../../data/IGRINS/SVC_log_labelled_csv.csv')
df_labelled = df_labelled.drop(columns=[df_labelled.columns[0]])

Yay! Let's read in the headers of the spectra.

## Spectra log

In [ ]:
keep_keywords = ['MJD-OBS', 'TELRA', 'TELDEC', 'AMSTART', 'OBJECT']

In [ ]:
fns = sorted(glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/**/SDCH*.fits'))

In [ ]:
df_log_spec = pd.DataFrame()
for i, fn in enumerate(fns):
    df = pd.DataFrame(get_header_info(fn, ext=0), index=[i])
    basename = os.path.basename(fn)
    df['fn'] = basename
    df_log_spec = df_log_spec.append(df)

In [ ]:
pd.options.display.max_rows = 999

In [ ]:
df_join = pd.concat([df_labelled, df_log_spec], axis=0, ignore_index=True)

Awesome! I just spot-checked the table and it will totally work.  Let's apply some pandas manipulation to condense the table into a publishable product.

In [ ]:
df_join = df_join.sort_values('MJD-OBS').reset_index(drop=True)

In [ ]:
from astropy.time import Time

In [ ]:
df_join['BTJD'] = Time(df_join['MJD-OBS'], format='mjd').btjd

In [ ]:
df_join = df_join.drop(columns=['MJD-OBS', 'TELRA', 'TELDEC', 'AMSTART', 'On Slit'])

Transfer the target labels from surrounding SVC images that were inspected by eye.

In [ ]:
df_join = df_join.bfill()

In [ ]:
df_join = df_join[df_join.fn.str.contains('SDCH')]

In [ ]:
df_join['Night'] = df_join.fn.str.split('_').apply(lambda x: x[1])

In [ ]:
df_join['File'] = df_join.fn.str.split('_').apply(lambda x: x[2]).str.split('.fits').apply(lambda x: x[0])

In [ ]:
cols = ['Night','Binary Component','File', 'Nod Position', 'BTJD',]

In [ ]:
df_join = df_join[cols]

In [ ]:
df_join.set_index(['Night', 'Binary Component'])

In [ ]:
df_final = df_join.set_index(['Night', 'File']).groupby(['Night', 'Binary Component']).BTJD.mean().to_frame()

In [ ]:
print(df_final.to_latex(header=True))

Let's trim the table to just the contents so that we can use the ApJ table header.

In [ ]:
df_final.reset_index().to_csv('../../data/IGRINS/2021A_metadata_log_CORRECT.csv', index=False)

In [ ]:
lines = df_final.to_latex(header=True).split('\n')[5:-3]
output = ''
for line in lines:
    output += (line + '\n')

In [ ]:
print(output)

In [ ]:
with open('../../document/paper1/IGRINS_obs_log.txt', 'w') as f:
    f.write(output)

### The end!